In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import shap
from pickle import dump
from pickle import load
from sklearn.preprocessing import OneHotEncoder
import keras_tuner as kt

In [9]:
!pip install -q -U keras-tuner

In [10]:
tf.__version__
tf.test.is_built_with_cuda()

True

In [11]:
X_train = pd.read_csv('X_train.csv')
X_val = pd.read_csv('X_val.csv')
y_train = pd.read_csv('y_train.csv')
y_val = pd.read_csv('y_val.csv')

In [12]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [17]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [18]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


In [19]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 31 Complete [00h 00m 07s]
val_accuracy: 0.012715713120996952

Best val_accuracy So Far: 0.6469119191169739
Total elapsed time: 00h 01m 41s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.01.



In [21]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
551/551 [==============================] - 1s 1ms/step - loss: 7.0365 - accuracy: 1.3957e-04 - val_loss: 8.6003 - val_accuracy: 0.0000e+00
Epoch 2/50
551/551 [==============================] - 1s 1ms/step - loss: 8.5985 - accuracy: 0.0000e+00 - val_loss: 9.0597 - val_accuracy: 0.0000e+00
Epoch 3/50
551/551 [==============================] - 1s 1ms/step - loss: 9.0555 - accuracy: 0.0000e+00 - val_loss: 9.0608 - val_accuracy: 0.0000e+00
Epoch 4/50
551/551 [==============================] - 1s 2ms/step - loss: 9.1360 - accuracy: 0.0000e+00 - val_loss: 9.5192 - val_accuracy: 0.0000e+00
Epoch 5/50
551/551 [==============================] - 1s 1ms/step - loss: 9.4756 - accuracy: 0.0000e+00 - val_loss: 9.5192 - val_accuracy: 0.0000e+00
Epoch 6/50
551/551 [==============================] - 1s 1ms/step - loss: 9.4834 - accuracy: 0.0000e+00 - val_loss: 9.5192 - val_accuracy: 0.0000e+00
Epoch 7/50
551/551 [==============================] - 1s 1ms/step - loss: 9.5037 - accuracy: 0.0000e

In [23]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

551/551 [==============================] - 1s 1ms/step - loss: 6.8062 - accuracy: 0.0283 - val_loss: 6.7600 - val_accuracy: 0.3531


In [24]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

NameError: name 'img_test' is not defined